In [1]:
import pymongo
import operator
import itertools
from pprint import pprint
from collections import OrderedDict

In [2]:
con = pymongo.MongoClient()
db = con.lacuerda
art = db.artists
ver = db.versions
songs = db.songs

def show_cursor(cursor, limit=10):
    for item in itertools.islice(cursor, limit):
        pprint(item)

### Usar slug del artista como _id

In [17]:
art.find_one()

{'_id': ObjectId('561bdcff6f3f6befd579cf85'),
 'genero': 'Música Religiosa',
 'nombre': 'Worship Together',
 'pais': 'us',
 'slug': 'worship_together'}

In [29]:
c=art.aggregate([
        {"$project": dict(_id=0, genero=1, slug=1, nombre=1, pais=1)},
        {"$project": dict(_id="$slug", genero=1, nombre=1, pais=1)},
        {"$out": "artists_with_id"}
    ])
show_cursor(c)

In [32]:
db.artists_with_id.find_one()

{'_id': 'worship_together',
 'genero': 'Música Religiosa',
 'nombre': 'Worship Together',
 'pais': 'us'}

In [34]:
art.drop()

In [36]:
db.artists_with_id.rename('artists')

In [37]:
db.artists.find_one()

{'_id': 'worship_together',
 'genero': 'Música Religiosa',
 'nombre': 'Worship Together',
 'pais': 'us'}

### Usar código de la canción como _id

In [38]:
keys = ver.find_one().keys()
keys

dict_keys(['codigo', '_id', 'acordes', 'transcriptor', 'autor', 'puntaje', 'nombre', 'version_number', 'formato', 'contenido', 'votos', 'slug', 'album', 'artista'])

In [43]:
proy = {key: 0 if key == '_id' else 1 for key in keys}
proy

{'_id': 0,
 'acordes': 1,
 'album': 1,
 'artista': 1,
 'autor': 1,
 'codigo': 1,
 'contenido': 1,
 'formato': 1,
 'nombre': 1,
 'puntaje': 1,
 'slug': 1,
 'transcriptor': 1,
 'version_number': 1,
 'votos': 1}

In [44]:
proy2 = proy.copy()
proy2['_id'] = '$codigo'
del proy2['codigo']
proy2

{'_id': '$codigo',
 'acordes': 1,
 'album': 1,
 'artista': 1,
 'autor': 1,
 'contenido': 1,
 'formato': 1,
 'nombre': 1,
 'puntaje': 1,
 'slug': 1,
 'transcriptor': 1,
 'version_number': 1,
 'votos': 1}

In [45]:
c=ver.aggregate([
        {"$project": proy},
        {"$project": proy2},
        {"$out": "versions_with_id"}
    ])
show_cursor(c)

In [47]:
db.versions_with_id.find_one({}, dict(acordes=0, contenido=0))

{'_id': 'nek0491',
 'album': 'Nuevas Direcciones (2009)',
 'artista': 'nek',
 'autor': 'Nek',
 'formato': 'R',
 'nombre': 'Una hora más',
 'puntaje': 9.25,
 'slug': 'una_hora_mas',
 'transcriptor': 'donyoyo',
 'version_number': 1,
 'votos': 4}

In [48]:
ver.drop()

In [49]:
db.versions_with_id.rename('versions')

### Crear índices

In [52]:
ver.create_index([("artista", 1), ("slug", 1), ("version_number", 1)], unique=True)

'artista_1_slug_1_version_number_1'

In [64]:
ver.find_one({},{"acordes":0,"contenido":0})

{'_id': 'nek0491',
 'album': 'Nuevas Direcciones (2009)',
 'artista': 'nek',
 'autor': 'Nek',
 'formato': 'R',
 'nombre': 'Una hora más',
 'puntaje': 9.25,
 'slug': 'una_hora_mas',
 'transcriptor': 'donyoyo',
 'version_number': 1,
 'votos': 4}

### Crear colección songs
Con las versiones correspondientes embebidas

In [65]:
# Estructura de la colección versiones
# No muestro los campos acordes ni contenido para verlo mejor
ver.find_one({}, {"acordes":0, "contenido": 0})

{'_id': 'nek0491',
 'album': 'Nuevas Direcciones (2009)',
 'artista': 'nek',
 'autor': 'Nek',
 'formato': 'R',
 'nombre': 'Una hora más',
 'puntaje': 9.25,
 'slug': 'una_hora_mas',
 'transcriptor': 'donyoyo',
 'version_number': 1,
 'votos': 4}

In [22]:
cur = ver.aggregate([
        {"$sort": OrderedDict([('artista', 1), ('slug', 1), ('album', -1)])},  # Para elegir un álbum que no sea None
        {"$group": {"_id": {"artista": "$artista",
                            "slug": "$slug"},
                    "nombre": {"$first": "$nombre"},
                    "album": {"$first": "$album"},
                    # "version_count": {"$sum": 1},
                    "versiones": {"$push": {"_id": "$_id", 
                                            "acordes": "$acordes",
                                            "contenido": "$contenido",
                                            "album": "$album",
                                            "autor": "$autor",
                                            "formato": "$formato",
                                            "puntaje": "$puntaje",
                                            "votos": "$votos",
                                            "transcriptor": "$transcriptor",
                                            "version_number": "$version_numer"}}}},
        {"$project": {"_id": {"$concat": ["$_id.artista", "/", "$_id.slug"]},
                      "artista": "$_id.artista",
                      "slug": "$_id.slug",
                      "nombre": "$nombre",
                      "album": "$album",
                      "versiones": "$versiones"}},
        # {"$sort": {"_id": -1}},
        # {"$limit": 10}
        {"$out": "songs"}
    ], allowDiskUse=True)
show_cursor(cur)

In [23]:
db.songs.count(), db.versions.count()

(120165, 162392)

### Índices comunes en songs

In [24]:
songs = db.songs

In [25]:
songs.create_index([('artista', 1), ('slug', 1)])

'artista_1_slug_1'

In [26]:
songs.create_index('versiones._id', unique=True)

'versiones._id_1'

### Nombre del artista embebido a la canción para crear índice compuesto de texto

In [29]:
# Estructura de la colección songs
# No muestro los campos acordes ni contenido para verlo mejor
songs.find_one({"_id": "andres_calamaro/cartas_sin_marcar"}, {"versiones.acordes":0, "versiones.contenido": 0})

{'_id': 'andres_calamaro/cartas_sin_marcar',
 'album': 'Por Mirarte (1988)',
 'artista': 'andres_calamaro',
 'nombre': 'Cartas sin marcar',
 'nombre_artista': 'Andrés Calamaro',
 'slug': 'cartas_sin_marcar',
 'versiones': [{'_id': 'acal0082',
   'album': 'Por Mirarte (1988)',
   'autor': 'Rot Calamaro',
   'formato': 'R',
   'puntaje': 9.48,
   'transcriptor': '44702',
   'votos': 86},
  {'_id': 'acal0083',
   'album': 'Por Mirarte (1988)',
   'autor': 'Rot Calamaro',
   'formato': 'K',
   'puntaje': 8.62,
   'transcriptor': '38683',
   'votos': 22},
  {'_id': 'acal0081',
   'album': 'Por Mirarte (1988)',
   'autor': 'Rot Calamaro',
   'formato': 'R',
   'puntaje': 8.71,
   'transcriptor': "javascript:amigo('N:0')",
   'votos': 52}]}

In [28]:
import time; init = time.time()
for artist in art.find():
    songs.update_many({"artista": artist['_id']}, {"$set": {"nombre_artista": artist['nombre']}})
print(time.time() - init)

34.808268547058105


### Crear text indexes en songs (nombre de canción, nombre de artista, álbum)

In [32]:
songs.create_index([("nombre", "text"), ("nombre_artista", "text"), ("album", "text")])

'nombre_text_nombre_artista_text_album_text'

In [31]:
songs.drop_index('slug_text_nombre_artista_text_album_text')

### Crear colección songs_search para mejorar performance
La búsqueda de texto en songs trae muy malos resultados en cuanto a performance. Voy a probar creando una colección similar pero solamente con los campos de búsqueda

In [4]:
cur=songs.aggregate([
        {"$project": dict(album=1, artista=1, nombre=1, nombre_artista=1, slug=1)},
        #{"$limit": 5},
        {"$out": "songs_search"}
    ])

In [6]:
songs_search = db.songs_search
songs_search.create_index?

In [10]:
songs_search.drop_index('nombre_text_nombre_artista_text_album_text')

In [14]:
songs_search.create_index([("nombre", "text"), ("nombre_artista", "text"), ("album", "text")],
                           default_language='spanish',
                           weights=dict(album=1, nombre_artista=2, nombre=4))

'nombre_text_nombre_artista_text_album_text'

In [ ]:
song